In [354]:
%reset

In [355]:
import psycopg2
import pandas as pd
import numpy as np

In [357]:
con=psycopg2.connect(
    host="hostname",
    port=15432,
    database="database",
    user="username",
    password="password",
)


In [358]:
querry_comm="""select * from communications"""
communications=pd.read_sql(querry_comm,con)
communications.head(5)

,communication_id,site_id,visitor_id,date_time
0,460453730,16649,615231236,2018-04-05 17:35:16
1,2092162,16649,658040990,2018-04-07 16:48:44
2,2061609,16649,848021975,2018-04-02 09:20:20
3,458670105,30374,1225592954,2018-04-02 18:38:03
4,2092238,16649,1235856228,2018-04-07 17:09:21


In [359]:
querry_session="""select * from sessions"""
sessions=pd.read_sql(querry_session,con)
sessions.head(5)

,visitor_session_id,site_id,visitor_id,date_time,campaign_id
0,2038323227,16649,1322630246,2018-04-03 22:39:12,44843
1,2043591207,16649,1325892110,2018-04-04 19:45:39,116083
2,2026999821,16649,1315616600,2018-04-02 08:56:25,116083
3,2038059953,30374,1322459808,2018-04-03 21:38:44,116085
4,2030418799,16649,1317685222,2018-04-02 17:02:44,116083


In [360]:
print(len(communications['visitor_id']),
      len(sessions['visitor_id']))

24 1177


In [361]:
%time
session_date_time_list=[]
row_n_list=[]
for i in range(len(communications['visitor_id'])):
    try:
        session_date_time_list.append(sessions[sessions['visitor_id']==communications['visitor_id'][i]]['date_time'][sessions['date_time']<=
        communications['date_time'][communications['visitor_id']==communications['visitor_id'][i]].iloc[0]].sort_values(ascending=False).iloc[1])
        row_n_list.append(len(sessions[sessions['visitor_id']==communications['visitor_id'][i]]['date_time'][sessions['date_time']<=
        communications['date_time'][communications['visitor_id']==communications['visitor_id'][i]].iloc[0]].sort_values(ascending=False).iloc[1:]))
    except IndexError:
        session_date_time_list.append(np.NaN)
        row_n_list.append(0)

Wall time: 0 ns


In [362]:
session_date_time_list

[nan,
 Timestamp('2018-04-07 12:23:22'),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 Timestamp('2018-04-03 12:54:24'),
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [363]:
session_date_time_list[13]

Timestamp('2018-04-03 12:54:24')

In [332]:
row_n_list[13]

1

In [333]:
communications['visitor_id'][13]

1320493028

In [364]:
joined=communications.join(sessions,how='left',lsuffix='x', rsuffix='y',on='visitor_id')
joined.head(5)

,communication_id,site_idx,visitor_idx,date_timex,visitor_session_id,site_idy,visitor_idy,date_timey,campaign_id
0,460453730,16649,615231236,2018-04-05 17:35:16,NaN,NaN,NaN,NaT,NaN
1,2092162,16649,658040990,2018-04-07 16:48:44,NaN,NaN,NaN,NaT,NaN
2,2061609,16649,848021975,2018-04-02 09:20:20,NaN,NaN,NaN,NaT,NaN
3,458670105,30374,1225592954,2018-04-02 18:38:03,NaN,NaN,NaN,NaT,NaN
4,2092238,16649,1235856228,2018-04-07 17:09:21,NaN,NaN,NaN,NaT,NaN


In [365]:
joined['row_n']=row_n_list
joined['row_n']

0     0
1     7
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    1
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: row_n, dtype: int64

In [367]:
joined['row_n']

0     0
1     7
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    1
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: row_n, dtype: int64

In [366]:
joined['sessions_date_time']=session_date_time_list
joined['sessions_date_time']

0                    NaT
1    2018-04-07 12:23:22
2                    NaT
3                    NaT
4                    NaT
5                    NaT
6                    NaT
7                    NaT
8                    NaT
9                    NaT
10                   NaT
11                   NaT
12                   NaT
13   2018-04-03 12:54:24
14                   NaT
15                   NaT
16                   NaT
17                   NaT
18                   NaT
19                   NaT
20                   NaT
21                   NaT
22                   NaT
23                   NaT
Name: sessions_date_time, dtype: datetime64[ns]

In [369]:
# joined.to_sql('JOINED',con=con,index=False)

In [370]:
joined.head(5)


,communication_id,site_idx,visitor_idx,date_timex,visitor_session_id,site_idy,visitor_idy,date_timey,campaign_id,row_n,sessions_date_time
0,460453730,16649,615231236,2018-04-05 17:35:16,NaN,NaN,NaN,NaT,NaN,0,NaT
1,2092162,16649,658040990,2018-04-07 16:48:44,NaN,NaN,NaN,NaT,NaN,7,2018-04-07 12:23:22
2,2061609,16649,848021975,2018-04-02 09:20:20,NaN,NaN,NaN,NaT,NaN,0,NaT
3,458670105,30374,1225592954,2018-04-02 18:38:03,NaN,NaN,NaN,NaT,NaN,0,NaT
4,2092238,16649,1235856228,2018-04-07 17:09:21,NaN,NaN,NaN,NaT,NaN,0,NaT


In [372]:
pd.read_sql("""select * from joined_sql""",con=con)


,communication_id,site_id,visitor_id,date_time,sessions_date_time,visitor_session_id,row_n
0,460453730,16649,615231236,2018-04-05 17:35:16,NaT,2048740059,0
1,2092162,16649,658040990,2018-04-07 16:48:44,2018-04-07 12:23:22,2054754818,7
2,2061609,16649,848021975,2018-04-02 09:20:20,NaT,2027105873,0
3,458670105,30374,1225592954,2018-04-02 18:38:03,NaT,2030992565,0
4,2092238,16649,1235856228,2018-04-07 17:09:21,NaT,2054740380,0
5,2092428,16649,1279156408,2018-04-07 18:21:16,NaT,2054991378,0
6,457537963,16649,1310647296,2018-04-01 10:00:28,NaT,2022247367,0
7,456995966,16649,1310647296,2018-04-01 10:16:50,NaT,2022247367,0
8,2056235,16649,1311935382,2018-04-01 00:21:49,NaT,2021383539,0
9,2056231,16649,1311935382,2018-04-01 00:21:07,NaT,2021383539,0


In [374]:
sql_script="""

create table from_py_sql as
(
    SELECT FRT.visitor_id,communication_id,site_id,row_n,campaign_id,FRT.sessions_date_time,date_time as communication_date_time from (
                    SELECT * from communications com
                    left join LATERAL
                    (
                        select LEAD(date_time) over () as sessions_date_time, visitor_session_id,campaign_id
                        from sessions
                        where (visitor_id = com.visitor_id)
                        and (site_id = com.site_id)
                        and (date_time<=com.date_time)
                        limit 1
                    ) re on com.visitor_id = visitor_id
                  ) FRT
    left join LATERAL (
                        select Count(date_time)-1 as row_n, visitor_id
                        from LATERAL (
                                        select date_time
                                        from sessions
                                        where (visitor_id = FRT.visitor_id)
                                        and (date_time <=frt.date_time)
                                     ) a where visitor_id = FRT.visitor_id and site_id = frt.site_id
                      ) sss
    on FRT.visitor_id = sss.visitor_id

)
"""


In [381]:
curs=con.cursor()


In [383]:
curs.execute(sql_script)
con.commit() # <--- makes sure the change is shown in the database
con.close()
curs.close()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
